# Tutorial Grid Search Optimization Algorithm

I've got some red pepper flakes up my ass to follow this tutorial :D

Grid Search Optimization Algorithm in Python: https://stackabuse.com/grid-search-optimization-algorithm-in-python/

Cross Validation and Grid Search for Model Selection in Python: https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/


## CrossValidation X GridSearch


* __Grid Search__ is an optimization (automation) algorithm, used for hyperparameter tunning in machine learning.


* __CrossValidation__ is intended to minimize the variance problem within a dataset. Sklearn provide a function to split the data into several trains and tests sets called K-Fold Cross-Validation, in which K is any number. K-1 will be used to train the data and the remaining one to test it. The algorithm is trained and then tested K times. Finally, the result of the K-Fold Cross-Validation is the average of the results obtained on each set

![image.png](https://s3.amazonaws.com/stackabuse/media/cross-validation-grid-search-model-selection-python-1.png)



In the second fold, SET A, SET B, SET C, and SET D are used for training and SET E is used as testing. The process continues until every set is at least used once for training and once for testing. The final result is the average of results obtained using all folds. This way we can get rid of the variance. Using standard deviation of the results obtained from each fold we can in fact find the variance in the overall result.



In [67]:
import sys # files
import numpy as np # linear algebra
import pandas as pd # tables

# preprocessing
from sklearn.preprocessing import StandardScaler # normalization
from sklearn.model_selection import train_test_split

# modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# grid search and kfold
from sklearn.model_selection import GridSearchCV, KFold

# Load data

In [41]:
columns = ['num_pregnant', 'glucose_concentration', 'blood_pressure', 'skin_thickness',
           'serum_insulin', 'BMI', 'pedigree_function', 'age', 'class']

data_path = "https://raw.githubusercontent.com/mkhalid1/Machine-Learning-Projects-Python-/master/Grid%20Search/pima-indians-diabetes.csv"

df = pd.read_csv(data_path, 
                 names=columns,
                 # skip the first rows as it only contains description
                 skiprows=9)

print(df.shape)
df.head(10)

(768, 9)


,num_pregnant,glucose_concentration,blood_pressure,skin_thickness,serum_insulin,BMI,pedigree_function,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
num_pregnant             768 non-null int64
glucose_concentration    768 non-null int64
blood_pressure           768 non-null int64
skin_thickness           768 non-null int64
serum_insulin            768 non-null int64
BMI                      768 non-null float64
pedigree_function        768 non-null float64
age                      768 non-null int64
class                    768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [43]:
# convert dataframe to numpy array
dataset = df.values

# Preprocess

In [48]:
# split data into X and Y
X = dataset[:, 0:8]
y = dataset[:, 8].astype(int)

# normalize the X data
# don't do that to y variables
scaler = StandardScaler().fit(X)

# transform the X variable
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.head()

,0,1,2,3,4,5,6,7
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [59]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y)

# Modeling

In [62]:
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    
    # first layer
    model.add(Dense(8, input_dim=8, 
                    kernel_initializer='normal',
                    activation='relu'
                   ))
    
    # second layer
    model.add(Dropout(dropout_rate))
    
    # third layer
    model.add(Dense(4, 
                    input_dim=8,
                    kernel_initializer='normal'
                   ))
    
    # forth layer
    model.add(Dropout(dropout_rate))
    
    # firth layer:
    # I GUESS: last layer shoudl present the activation function
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr=learn_rate)
    model.compile(loss='binary_crossentropy', 
                  optimizer=adam,
                  metrics=['accuracy'])
    
    return model
    

## Train the model without Grid Search

In [64]:
# declare the parameters
dropout_rate = 0.1
epochs = 1
batch_size = 20
learn_rate = 0.001

# create the model 
model = create_model(learn_rate, dropout_rate)

# fit the model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1
         )

29/29 [==============================] - 0s 2ms/step - loss: 0.6864 - accuracy: 0.6111


# Optimizing Hyper-parameters using Grid Search

In [71]:
# create the model
model = KerasClassifier(build_fn=create_model, verbose=1)

In [77]:
# define the parameters to be tested using Grid Search
learn_rate = [0.001, 0.02, 0.2]
dropout_rate = [0.0, 0.2, 0.4]
batch_size = [10, 20, 30]
epochs = [1, 2, 3]

seed = 42

# make a dictionary of the grip search parameters
param_grid = dict(learn_rate=learn_rate, 
                  dropout_rate=dropout_rate,
                  batch_size=batch_size,
                  epochs=epochs
                 )

In [ ]:
# build the GridSearchCV
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid,
                    cv=KFold(random_state=seed),
                    verbose=10
                   )


In [78]:
# fit the grid search
grid_search = grid.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] batch_size=10, dropout_rate=0.0, epochs=1, learn_rate=0.001 .....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


12/12 [==============================] - 0s 1ms/step - loss: 0.6788 - accuracy: 0.5948
[CV]  batch_size=10, dropout_rate=0.0, epochs=1, learn_rate=0.001, score=0.595, total=   1.0s
[CV] batch_size=10, dropout_rate=0.0, epochs=1, learn_rate=0.001 .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


12/12 [==============================] - 0s 1ms/step - loss: 0.6823 - accuracy: 0.5913
[CV]  batch_size=10, dropout_rate=0.0, epochs=1, learn_rate=0.001, score=0.591, total=   1.0s
[CV] batch_size=10, dropout_rate=0.0, epochs=1, learn_rate=0.001 .....


...

4/4 [==============================] - 0s 2ms/step - loss: 0.5698 - accuracy: 0.6466
[CV]  batch_size=30, dropout_rate=0.2, epochs=3, learn_rate=0.2, score=0.647, total=   1.2s
[CV] batch_size=30, dropout_rate=0.2, epochs=3, learn_rate=0.2 .......
Epoch 1/3
16/16 [==============================] - 0s 7ms/step - loss: 0.6191 - accuracy: 0.6768
Epoch 2/3
16/16 [==============================] - 0s 9ms/step - loss: 0.5562 - accuracy: 0.6941
Epoch 3/3
4/4 [==============================] - 0s 2ms/step - loss: 0.5846 - accuracy: 0.7826
[CV]  batch_size=30, dropout_rate=0.2, epochs=3, learn_rate=0.2, score=0.783, total=   1.3s
[CV] batch_size=30, dropout_rate=0.2, epochs=3, learn_rate=0.2 .......
Epoch 1/3
16/16 [==============================] - 0s 2ms/step - loss: 0.6780 - accuracy: 0.6356
Epoch 2/3
16/16 [==============================] - 0s 2ms/step - loss: 0.6003 - accuracy: 0.6638
Epoch 3/3
4/4 [==============================] - 0s 3ms/step - loss: 0.5371 - accuracy: 0.7130
[CV]  batch

Epoch 1/2
16/16 [==============================] - 0s 2ms/step - loss: 0.6882 - accuracy: 0.6030
Epoch 2/2
4/4 [==============================] - 0s 2ms/step - loss: 0.6684 - accuracy: 0.6783
[CV]  batch_size=30, dropout_rate=0.4, epochs=2, learn_rate=0.001, score=0.678, total=   1.3s
[CV] batch_size=30, dropout_rate=0.4, epochs=2, learn_rate=0.001 .....
Epoch 1/2
16/16 [==============================] - 0s 2ms/step - loss: 0.6923 - accuracy: 0.5705
Epoch 2/2
4/4 [==============================] - 0s 2ms/step - loss: 0.6819 - accuracy: 0.6348
[CV]  batch_size=30, dropout_rate=0.4, epochs=2, learn_rate=0.001, score=0.635, total=   1.3s
[CV] batch_size=30, dropout_rate=0.4, epochs=2, learn_rate=0.001 .....
Epoch 1/2
16/16 [==============================] - 0s 2ms/step - loss: 0.6905 - accuracy: 0.6486
Epoch 2/2
4/4 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.6870
[CV]  batch_size=30, dropout_rate=0.4, epochs=2, learn_rate=0.001, score=0.687, total=   1.0s


4/4 [==============================] - 0s 2ms/step - loss: 0.6586 - accuracy: 0.6348
[CV]  batch_size=30, dropout_rate=0.4, epochs=3, learn_rate=0.001, score=0.635, total=   1.2s
[CV] batch_size=30, dropout_rate=0.4, epochs=3, learn_rate=0.001 .....
Epoch 1/3
16/16 [==============================] - 0s 2ms/step - loss: 0.6883 - accuracy: 0.6725
Epoch 2/3
16/16 [==============================] - 0s 2ms/step - loss: 0.6807 - accuracy: 0.6551
Epoch 3/3
4/4 [==============================] - 0s 1ms/step - loss: 0.6605 - accuracy: 0.6870
[CV]  batch_size=30, dropout_rate=0.4, epochs=3, learn_rate=0.001, score=0.687, total=   1.2s
[CV] batch_size=30, dropout_rate=0.4, epochs=3, learn_rate=0.02 ......
Epoch 1/3
16/16 [==============================] - 0s 2ms/step - loss: 0.6433 - accuracy: 0.6565
Epoch 2/3
16/16 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.7283
Epoch 3/3
4/4 [==============================] - 0s 2ms/step - loss: 0.5457 - accuracy: 0.6983
[CV]  b

[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed: 11.8min finished


Epoch 1/3
58/58 [==============================] - 0s 3ms/step - loss: 0.5672 - accuracy: 0.6927
Epoch 2/3
58/58 [==============================] - 0s 5ms/step - loss: 0.5112 - accuracy: 0.7361
Epoch 3/3
58/58 [==============================] - 0s 4ms/step - loss: 0.5059 - accuracy: 0.7760


In [81]:
print("Best: {0}, using {1}".format(grid_search.best_score_, 
                                    grid_search.best_params_))

Best: 0.7691754102706909, using {'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 3, 'learn_rate': 0.02}


In [82]:
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']


In [83]:
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))


0.6389655232429504 (0.04021782719233177) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 1, 'learn_rate': 0.001}
0.758815586566925 (0.04060271316718443) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 1, 'learn_rate': 0.02}
0.7170014977455139 (0.055487791922429607) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 1, 'learn_rate': 0.2}
0.6476611733436585 (0.05390534047135536) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 2, 'learn_rate': 0.001}
0.7396851658821106 (0.03290953065600495) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 2, 'learn_rate': 0.02}
0.6857271313667297 (0.06979943268588795) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 2, 'learn_rate': 0.2}
0.741514253616333 (0.06837278732222116) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 3, 'learn_rate': 0.001}
0.760509729385376 (0.027912095147361086) with: {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 3, 'learn_rate': 0.02}
0.6685757040977478 (0.0778517016525707) 